<a href="https://colab.research.google.com/github/krushnajayale/Linguist-Pro/blob/main/GenZ_translator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Install the missing library and other project requirements
!pip install pyngrok streamlit transformers torch sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 78.0 MB/s eta 0:00:00


In [35]:
%%writefile app.py
import streamlit as st
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

# ===============================
# 1. DATA DICTIONARIES
# ===============================
MORSE_CODE_DICT = {
    'A': '.-', 'B': '-...', 'C': '-.-.', 'D': '-..', 'E': '.', 'F': '..-.',
    'G': '--.', 'H': '....', 'I': '..', 'J': '.---', 'K': '-.-', 'L': '.-..',
    'M': '--', 'N': '-.', 'O': '---', 'P': '.--.', 'Q': '--.-', 'R': '.-.',
    'S': '...', 'T': '-', 'U': '..-', 'V': '...-', 'W': '.--', 'X': '-..-',
    'Y': '-.--', 'Z': '--..', '1': '.----', '2': '..---', '3': '...--',
    '4': '....-', '5': '.....', '6': '-....', '7': '--...', '8': '---..',
    '9': '----.', '0': '-----', ',': '--..--', '.': '.-.-.-', '?': '..--..',
    '/': '-..-.', '-': '-....-', '(': '-.--.', ')': '-.--.-', ' ': '/'
}

GENZ_DICT = {
    "ate": "did something extremely well", "ate and left no crumbs": "performed perfectly",
    "af": "very / extremely", "amp": "excited or hyped", "atp": "at this point",
    "atm": "at the moment", "audacity": "bold or shocking behavior",
    "bet": "agree / okay / sure", "big yikes": "very embarrassing",
    "boujee": "fancy or high-maintenance", "bussing": "extremely good (food)",
    "built different": "uniquely talented", "brainrot": "obsessed with something",
    "cap": "lie", "no cap": "no lie / truth", "cringe": "embarrassing",
    "caught in 4k": "caught clearly", "chief": "friend / bro", "cooked": "ruined / finished",
    "cook": "perform very well", "delulu": "delusional in a funny way",
    "drip": "good fashion sense", "down bad": "emotionally desperate",
    "deadass": "seriously", "defo": "definitely", "dont play": "don’t mess with me",
    "dubs only": "only wins", "era": "phase of life", "extra": "over the top",
    "eating": "doing great", "emo hours": "emotional mood", "emotional damage": "embarrassment",
    "flex": "show off", "fomo": "fear of missing out", "facts": "true / agree",
    "finesse": "handle skillfully", "fake flex": "pretending to be rich",
    "ghost": "stop replying suddenly", "goat": "greatest of all time",
    "gyatt": "expression of surprise", "gigachad": "extremely confident person",
    "going off": "speaking strongly", "glizzy": "hotdog", "goofer": "silly person",
    "highkey": "openly", "hop off": "stop bothering", "hard launch": "reveal relationship",
    "headass": "goofy person", "hits different": "feels unique", "im dead": "very funny",
    "its giving": "gives a certain vibe", "iykyk": "if you know you know",
    "ick": "instant dislike", "in my bag": "focused and productive",
    "jit": "kid / childish person", "jaw dropped": "shocked", "jinxed": "bad luck",
    "joever": "it’s over", "karen": "entitled rude person", "kinda ate": "did good",
    "kmsl": "laughing very hard", "knuck if you buck": "ready to fight",
    "lowkey": "slightly / secretly", "loki": "low-key", "l take": "bad opinion",
    "lit": "fun or exciting", "left on read": "seen but ignored", "locked in": "fully focused",
    "mid": "average", "main character energy": "acting like the hero",
    "manifest": "strongly wish for", "moots": "mutual followers", "man down": "defeated",
    "npc": "boring person", "no thoughts head empty": "clueless", "naur": "dramatic no",
    "ngl": "not gonna lie", "not the": "sarcastic reaction", "on god": "seriously",
    "ong": "on god", "oof": "expression of discomfort", "opp": "enemy", "over it": "fed up",
    "pookie": "cute nickname", "pressed": "upset or bothered", "pushin p": "doing it right",
    "periodt": "end of discussion", "pluh": "approval sound", "pov": "point of view",
    "quiet luxury": "minimal rich aesthetic", "quirked up": "energetic goofy vibe",
    "rizz": "flirting skill", "rizzless": "no charm", "ratioed": "fewer likes",
    "rent free": "stuck in mind", "real one": "genuine person", "receipts": "proof",
    "slay": "do amazing", "sigma": "confident person", "sus": "suspicious",
    "simp": "tries too hard", "snatched": "looks perfect", "swag": "cool style",
    "sussy baka": "silly suspicious person", "skibidi": "nonsense meme",
    "tea": "gossip", "tapped in": "well informed", "thats wild": "unbelievable",
    "thirsty": "attention-seeking", "toxic": "negative", "treat": "pamper",
    "understood the assignment": "perfectly done", "up": "successful",
    "unalive": "defeated", "vibes": "mood", "valid": "makes sense", "viral": "trending",
    "w": "win", "w take": "good opinion", "w move": "smart decision", "w rizz": "charm",
    "womp womp": "mocking disappointment", "yap": "talk too much", "yeet": "throw / excitement",
    "yk": "you know", "you ate": "you did great", "yassify": "glamorize",
    "zesty": "playful energy", "zoomies": "burst of energy"
}

LANG_MAP = {
    "English": "eng_Latn",
    "Hindi": "hin_Deva",
    "Marathi": "mar_Deva",
    "French": "fra_Latn",
    "Spanish": "spa_Latn",
    "German": "deu_Latn",
    "Japanese": "jpn_Jpan"
}

# ===============================
# 2. AI MODEL LOADING (FIXED LOGIC)
# ===============================
@st.cache_resource
def load_models():
    model_name = "facebook/nllb-200-distilled-600M"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    return tokenizer, model

# ===============================
# 3. PAGE CONFIG & CYBER THEME
# ===============================
st.set_page_config(page_title="GenZ & Morse Translator", layout="centered")

st.markdown("""
<style>
    .stApp {
        background: radial-gradient(circle at center, rgba(0,255,255,0.15), rgba(0,0,0,1)),
                    url('https://www.transparenttextures.com/patterns/cubes.png');
        background-color: #000;
        color: #fff;
        background-size: cover;
        background-position: center;
        background-repeat: no-repeat;
    }
    .cyber-title {
        text-align: center;
        font-size: 32px;
        font-weight: bold;
        text-shadow: 0 0 15px cyan;
        color: cyan !important;
        margin-bottom: 20px;
        letter-spacing: 2px;
    }
    label p { color: cyan !important; text-shadow: 0 0 5px cyan; font-size: 18px !important; }
    textarea, select {
        background-color: black !important;
        color: cyan !important;
        border: 1px solid cyan !important;
        box-shadow: 0 0 10px cyan !important;
        border-radius: 10px !important;
    }
    .stButton>button {
        width: 100%;
        background: cyan !important;
        color: black !important;
        font-weight: bold !important;
        font-size: 18px !important;
        box-shadow: 0 0 20px cyan !important;
        border: none !important;
    }
    .output-terminal {
        background-color: #001a1a !important;
        color: #00FF41 !important;
        padding: 15px;
        border-radius: 10px;
        border: 1px solid cyan;
        font-family: 'Courier New', monospace;
        margin-top: 15px;
        min-height: 50px;
    }
</style>
""", unsafe_allow_html=True)
# ===============================
# 4. APP NAVIGATION
# ===============================
if 'mode' not in st.session_state:
    st.session_state.mode = "GEN-Z"

if st.sidebar.button("🔄 SWITCH TRANSLATOR"):
    st.session_state.mode = "MORSE" if st.session_state.mode == "GEN-Z" else "GEN-Z"
    st.session_state.pop('output', None)
    st.rerun()

st.markdown("<div class='clean-container'>", unsafe_allow_html=True)

if st.session_state.mode == "GEN-Z":
    st.markdown("<div class='cyber-title'>Gen-Z Translator</div>", unsafe_allow_html=True)

    # ANY TO ANY SELECTION
    col1, col2 = st.columns(2)
    with col1:
        src_lang = st.selectbox("Source Language", list(LANG_MAP.keys()), index=0)
    with col2:
        tgt_lang = st.selectbox("Target Language", list(LANG_MAP.keys()), index=1)

    input_text = st.text_area("Input Text:", placeholder="Enter text to translate...")
    apply_slang = st.checkbox("Apply Gen-Z Slang ")

    if st.button("🚀 EXECUTE "):
        if input_text:
            with st.spinner("AI Processing..."):
                tokenizer, model = load_models()

                # Setup language codes
                src_code = LANG_MAP[src_lang]
                tgt_code = LANG_MAP[tgt_lang]

                # Tokenize with source language
                tokenizer.src_lang = src_code
                inputs = tokenizer(input_text, return_tensors="pt")

                # Generate in target language
                target_lang_id = tokenizer.convert_tokens_to_ids(tgt_code)
                generated = model.generate(**inputs, forced_bos_token_id=target_lang_id, max_length=200)
                output = tokenizer.decode(generated[0], skip_special_tokens=True)

                # Optional Slang Filter
                if apply_slang:
                    words = output.lower().split()
                    output = " ".join([GENZ_DICT.get(w.strip(".,!?"), w) for w in words])

                st.session_state.output = output
        else: st.warning("Input required.")

else:
    st.markdown("<div class='cyber-title'>Morse Translator</div>", unsafe_allow_html=True)

    # Sub-mode selection for Morse
    morse_action = st.radio("Action:", ["Encode (Text to Morse)", "Decode (Morse to Text)"], horizontal=True)

    input_text = st.text_area("Enter Text or Morse Code:", placeholder="... --- ..." if morse_action == "Decode" else "Hello")

    if st.button("🛡️ EXECUTE"):
        if input_text:
            if morse_action == "Encode (Text to Morse)":
                # Encode Logic
                res = " ".join([MORSE_CODE_DICT.get(c.upper(), '?') for c in input_text])
                st.session_state.output = res
            else:
                # Decode Logic
                # Split by space to get individual dots/dashes
                words = input_text.split(" / ") # Slashes represent word spaces
                decoded_message = []
                for word in words:
                    letters = word.split()
                    decoded_word = "".join([REVERSE_MORSE_DICT.get(l, '?') for l in letters])
                    decoded_message.append(decoded_word)
                st.session_state.output = " ".join(decoded_message)
        else:
            st.warning("Please enter input.")
# 5. SHARED OUTPUT
if 'output' in st.session_state:
    st.markdown("<br><label>System Output:</label>", unsafe_allow_html=True)
    st.markdown(f"<div class='output-terminal'>{st.session_state.output}</div>", unsafe_allow_html=True)

st.markdown("</div>", unsafe_allow_html=True)

Overwriting app.py


In [33]:
!pkill ngrok

In [36]:
from pyngrok import ngrok
import os

# 1. Setup ngrok authtoken
# Note: Ensure this token is valid from your ngrok dashboard
NGROK_AUTH_TOKEN = "37T04ZcKwd4sYFMCXZtaOjFFOOY_31t8dTZk8nkR7UppVF2Ka"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# 2. Run Streamlit in the background
# This starts the app.py file we created in the previous step
os.system("streamlit run app.py &")

# 3. Create the Tunnel
# This generates the public URL for your project submission
try:
    public_url = ngrok.connect(8501)
    print(f"✅ Your Project is Live at: {public_url}")
except Exception as e:
    print(f"❌ Error connecting to ngrok: {e}")

✅ Your Project is Live at: NgrokTunnel: "https://acyclic-hortensia-unwatched.ngrok-free.dev" -> "http://localhost:8501"
